In [1]:
# Author : Saurabh

In [2]:
# git@github.com:analyticsrepo01/llm_dedepulication.git

### basic setup

In [5]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [7]:
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)
from google.cloud import aiplatform_v1beta1, aiplatform
from google.protobuf import struct_pb2
import numpy as np

from tenacity import retry, stop_after_attempt, wait_random_exponential


In [9]:
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)
embedding_model =TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")


## Dedup logic

### data cleaning

In [11]:
import pandas as pd

# Load DataFrames
df1 = pd.read_csv('database1.csv')
df2 = pd.read_csv('database2.csv')

# Standardize Company Names
def standardize_name(name):
    name = name.lower()  # Lowercase
    name = name.replace('.', '').replace(',', '')  # Remove punctuation
    name = name.replace('limited', 'ltd')  # Abbreviation
    return name.strip()  # Remove extra spaces

df1['customer_name'] = df1['customer_name'].astype(str).apply(standardize_name)
df2['customer_name'] = df2['customer_name'].astype(str).apply(standardize_name)

# Standardize Phone Numbers
def standardize_phone(phone):
    phone = phone.replace('-', '').replace('(', '').replace(')', '').replace(' ', '')  # Remove formatting
    if phone.startswith('+1'):  # Remove +1 from US numbers
        phone = phone[2:]
    return phone

df1['customer_phone_number'] = df1['customer_phone_number'].astype(str).apply(standardize_phone)
df2['customer_phone_number'] = df2['customer_phone_number'].astype(str).apply(standardize_phone)

# Standardize Addresses (Simplified)
def standardize_address(address):
    address = address.lower()  # Lowercase
    address = address.replace('.', '').replace(',', '')  # Remove some punctuation
    address = address.replace('street', 'st').replace('avenue', 'ave')  # Abbreviate
    return address.strip()

df1['customer_address'] = df1['customer_address'].astype(str).apply(standardize_address)
df2['customer_address'] = df2['customer_address'].astype(str).apply(standardize_address)

# Standardize Directors (Optional)
def standardize_directors(directors):
    if pd.isna(directors):  # Handle missing values
        return ""
    directors = directors.lower()
    directors = directors.replace(' ', '')  # Remove spaces
    return directors.split(',')  # Split into a list

df1['customer_directors'] = df1['customer_directors'].astype(str).apply(standardize_directors)
df2['customer_directors'] = df2['customer_directors'].astype(str).apply(standardize_directors)


# ... (you can apply similar logic to standardize emails and other fields)

# Create a concatenated field for similarity search
df1['concatenated_field'] = df1['customer_name'] + ' ' + df1['customer_address'] 
df2['concatenated_field'] = df2['customer_name'] + ' ' + df2['customer_address']


In [12]:
df1.head()

,customer_name,customer_email,customer_phone_number,customer_directors,customer_address,concatenated_field
0,acme corporation,[email address removed],5551234567,"[johnsmith, janedoe]",123 main st anytown usa,acme corporation 123 main st anytown usa
1,globex solutions,[email address removed],5555558901,[robertjohnson],456 elm ave cityville,globex solutions 456 elm ave cityville
2,innovative tech ltd,[email address removed],+442079460123,"[emilydavis, michaelbrown]",789 oak ln london,innovative tech ltd 789 oak ln london
3,the coffee house,[email address removed],5555552345,[nan],101 java st coffeeville,the coffee house 101 java st coffeeville
4,bob's plumbing services,[email address removed],5559876543,[bobjones],202 pipe dr plumberton,bob's plumbing services 202 pipe dr plumberton


In [14]:
import re
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

# Download stopwords if you haven't already
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def tokenize_and_create_keyword_set(text):
    """
    Tokenizes text, removes stopwords, and creates a set of keywords.
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in tokens if not word in stop_words]
    return set(keywords)  # Use a set for efficient comparison later


[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
from sklearn.metrics.pairwise import cosine_similarity
# ... (other imports for text embedding model, keyword matching)

# Generate embeddings
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")
df1['embedding'] = df1['concatenated_field'].apply(lambda x: embedding_model.get_embeddings([x])[0].values)
df2['embedding'] = df2['concatenated_field'].apply(lambda x: embedding_model.get_embeddings([x])[0].values)

# Calculate cosine similarity
similarities = cosine_similarity(df1['embedding'].tolist(), df2['embedding'].tolist())

# Keyword matching
df1['keywords'] = df1['concatenated_field'].apply(tokenize_and_create_keyword_set)
df2['keywords'] = df2['concatenated_field'].apply(tokenize_and_create_keyword_set)



### calculate keyword overlap scores


In [16]:
# ... (previous code for loading, cleaning, and keyword set creation)

# Calculate Keyword Overlap Scores
def calculate_keyword_overlap(keywords1, keywords2):
    """
    Calculates set intersection and Jaccard similarity for two keyword sets.
    """
    intersection = len(keywords1.intersection(keywords2))
    union = len(keywords1.union(keywords2))
    jaccard_sim = intersection / union if union > 0 else 0  # Avoid division by zero
    return intersection, jaccard_sim

# Create Empty Columns for Scores
df1['intersection_score'] = 0
df1['jaccard_score'] = 0
df2['intersection_score'] = 0
df2['jaccard_score'] = 0

# Calculate Scores for Each Pair
for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        intersection, jaccard = calculate_keyword_overlap(row1['keywords'], row2['keywords'])
        df1.at[i, 'intersection_score'] = max(df1.at[i, 'intersection_score'], intersection)
        df1.at[i, 'jaccard_score'] = max(df1.at[i, 'jaccard_score'], jaccard)
        df2.at[j, 'intersection_score'] = max(df2.at[j, 'intersection_score'], intersection)
        df2.at[j, 'jaccard_score'] = max(df2.at[j, 'jaccard_score'], jaccard)

# Print the DataFrames with Scores
print(df1[['customer_name', 'intersection_score', 'jaccard_score']])
print(df2[['customer_name', 'intersection_score', 'jaccard_score']])


             customer_name  intersection_score  jaccard_score
0         acme corporation                   6       0.750000
1         globex solutions                   6       0.857143
2      innovative tech ltd                   4       0.444444
3         the coffee house                   5       0.714286
4  bob's plumbing services                   4       0.400000
          customer_name  intersection_score  jaccard_score
0             acme corp                   6       0.750000
1  globex solutions llc                   6       0.857143
2        innovatech ltd                   4       0.444444
3          coffee house                   5       0.714286
4    bob jones plumbing                   4       0.400000


/var/tmp/ipykernel_42068/2885201816.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.at[i, 'jaccard_score'] = max(df1.at[i, 'jaccard_score'], jaccard)
/var/tmp/ipykernel_42068/2885201816.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df2.at[j, 'jaccard_score'] = max(df2.at[j, 'jaccard_score'], jaccard)


### Combine scores and identify duplicates


In [17]:
# ... (previous code for loading, cleaning, embedding generation, and keyword overlap calculation)

# Combine Scores and Identify Duplicates
THRESHOLD_SIMILARITY = 0.85   # Adjust based on your data
THRESHOLD_JACCARD = 0.5       # Adjust based on your data

potential_duplicates = []

for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        if similarities[i][j] >= THRESHOLD_SIMILARITY and row1['jaccard_score'] >= THRESHOLD_JACCARD:
            potential_duplicates.append((row1['customer_name'], row2['customer_name'], similarities[i][j], row1['jaccard_score']))

# Create a DataFrame of Potential Duplicates
potential_duplicates_df = pd.DataFrame(potential_duplicates, columns=['customer_name_1', 'customer_name_2', 'similarity_score', 'jaccard_score'])

# Print Potential Duplicates
print(potential_duplicates_df)


    customer_name_1       customer_name_2  similarity_score  jaccard_score
0  acme corporation             acme corp          0.994756       0.750000
1  globex solutions  globex solutions llc          0.991921       0.857143
2  the coffee house          coffee house          0.925139       0.714286


### Hybrid Approach for Duplicate Identification


In [18]:
# ... (previous code for loading, cleaning, embedding generation, and keyword overlap calculation)

# Hybrid Approach for Duplicate Identification
def is_potential_duplicate(sim_score, jaccard_score):
    """
    Determines if a pair of records is a potential duplicate based on a hybrid approach.
    """
    # Weighted average with emphasis on similarity
    combined_score = 0.7 * sim_score + 0.3 * jaccard_score
    
    # Additional rules for handling specific cases (optional)
    if sim_score > 0.9 and jaccard_score > 0.2:  # High similarity, moderate keywords
        return True
    elif sim_score > 0.8 and jaccard_score > 0.4:  # Moderate similarity, good keywords
        return True
    elif combined_score > 0.65:                     # Overall decent score
        return True
    else:
        return False

# Identify Potential Duplicates
potential_duplicates = []
for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        if is_potential_duplicate(similarities[i][j], row1['jaccard_score']):
            potential_duplicates.append((row1['customer_name'], row2['customer_name'], similarities[i][j], row1['jaccard_score']))

# Create a DataFrame of Potential Duplicates
potential_duplicates_df = pd.DataFrame(potential_duplicates, columns=['customer_name_1', 'customer_name_2', 'similarity_score', 'jaccard_score'])

# Print Potential Duplicates
print(potential_duplicates_df)


            customer_name_1       customer_name_2  similarity_score  \
0          acme corporation             acme corp          0.994756   
1          acme corporation  globex solutions llc          0.651831   
2          acme corporation        innovatech ltd          0.618742   
3          acme corporation          coffee house          0.673708   
4          acme corporation    bob jones plumbing          0.608970   
5          globex solutions             acme corp          0.632943   
6          globex solutions  globex solutions llc          0.991921   
7          globex solutions        innovatech ltd          0.639730   
8          globex solutions          coffee house          0.600899   
9          globex solutions    bob jones plumbing          0.630439   
10      innovative tech ltd        innovatech ltd          0.926730   
11         the coffee house             acme corp          0.689812   
12         the coffee house  globex solutions llc          0.661415   
13    

In [43]:
# ... (previous code for loading, cleaning, embedding generation, keyword overlap calculation, and duplicate identification)

# Create Sets of Duplicate IDs
duplicate_ids_df1 = set(potential_duplicates_df['customer_name_1'])
duplicate_ids_df2 = set(potential_duplicates_df['customer_name_2'])

# Combine Non-Duplicate Entries
non_duplicate_df1 = df1[~df1['customer_name'].isin(duplicate_ids_df1)].copy()
non_duplicate_df2 = df2[~df2['customer_name'].isin(duplicate_ids_df2)].copy()
combined_non_duplicates_df = pd.concat([non_duplicate_df1, non_duplicate_df2])


# Add a Flag for Potential Duplicates
potential_duplicates_df["is_potential_duplicate"] = True

# Combine with Potential Duplicates
final_df = pd.concat([combined_non_duplicates_df, potential_duplicates_df])

# Remove Unnecessary Columns (Optional)
final_df = final_df.drop(columns=['embedding', 'keywords', 'intersection_score', 'jaccard_score'])

# Print Final DataFrame
print(final_df)


   customer_name customer_email customer_phone_number customer_directors  \
0            NaN            NaN                   NaN                NaN   
1            NaN            NaN                   NaN                NaN   
2            NaN            NaN                   NaN                NaN   
3            NaN            NaN                   NaN                NaN   
4            NaN            NaN                   NaN                NaN   
5            NaN            NaN                   NaN                NaN   
6            NaN            NaN                   NaN                NaN   
7            NaN            NaN                   NaN                NaN   
8            NaN            NaN                   NaN                NaN   
9            NaN            NaN                   NaN                NaN   
10           NaN            NaN                   NaN                NaN   
11           NaN            NaN                   NaN                NaN   
12          